In [2]:
!pip install neptune 
!pip install lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.1/448.1 kB 12.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 58.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.76
    Uninstalling botocore-1.29.76:
      Successfully uninstalled botocore-1.29.76
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.5.0 requires botocore<1.29.77,>=1.29.76, but you have botocore 1.29.133 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.4/562.4 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import numpy as np
import os
import torch
import torchvision
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, random_split
import lightning as L
import neptune.new as neptune
from torchmetrics import Accuracy
from torchmetrics import AveragePrecision
import os
import pandas as pd
from lightning.pytorch.callbacks import TQDMProgressBar
import math
import random

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/tmp/ipykernel_34/3539159077.py:9: NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.
  import neptune.new as neptune


# Data pre proccessing

In [13]:
def norm(data):
    return (data - data.min()) / (data.max() - data.min())

In [14]:
from sklearn.preprocessing import LabelEncoder
p = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/'
train_dir = p + '/train'

path = f'{train_dir}/tdcsfog'
files = [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.csv')]
tdcsfog_df = pd.concat([pd.read_csv(f).assign(Id=f.split('/')[-1].split('.')[0]) for f in files])
tdcsfog_df.index=range(len(tdcsfog_df))

path = f'{train_dir}/defog'
files = [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.csv')]
defog_df = pd.concat([pd.read_csv(f).assign(Id=f.split('/')[-1].split('.')[0]) for f in files])
defog_df.index=range(len(defog_df))
# normalize 
defog_df['AccV']=defog_df['AccV']/9.8
defog_df['AccML']=defog_df['AccML']/9.8
defog_df['AccAP']=defog_df['AccAP']/9.8

defog_df['AccV']=norm(defog_df['AccV'])
defog_df['AccML']=norm(defog_df['AccML'])
defog_df['AccAP']=norm(defog_df['AccAP'])

tdcsfog_df['AccV']=norm(tdcsfog_df['AccV'])
tdcsfog_df['AccML']=norm(tdcsfog_df['AccML'])
tdcsfog_df['AccAP']=norm(tdcsfog_df['AccAP'])


df = pd.concat([tdcsfog_df,defog_df])

tdcsfog_metadata = pd.read_csv(p+'/tdcsfog_metadata.csv')
defog_metadata = pd.read_csv(p+'/defog_metadata.csv')
metadata= pd.concat([tdcsfog_metadata,defog_metadata])
df = df.merge(metadata, on='Id', how='inner')
encoder = LabelEncoder()
df['Id']=encoder.fit_transform(df['Id'])

In [15]:
def get_idx_of_lables(df):
    any_ones = df[['StartHesitation', 'Turn', 'Walking']].eq(1).any(axis=1)
    idx_with_ones = df[any_ones].index
    return idx_with_ones

def wind(df, window_size_range):
    lable_indexes = list(get_idx_of_lables(df))
    times = np.array(df['Time'])

    lable_indexes = np.array(lable_indexes)
    mask = np.where(lable_indexes[1:] - lable_indexes[:-1] != 1)[0]

    start_segments = lable_indexes[np.concatenate((np.array([0]), mask + 1))]
    end_segments = lable_indexes[np.concatenate((mask, np.array([len(lable_indexes) - 1])))]

    res = []

    for start, end in zip(start_segments, end_segments):
        window_size = random.randint(int(window_size_range*0.8), int(window_size_range*1.2))
        res.extend(times[max(0, start - window_size // 2):start])
        res.extend(lable_indexes[start:end+1])
        res.extend(times[end:min(len(times)-1, end + window_size // 4)])

    return res

def window(df, window_size):
    win_has_label_idx = wind(df, window_size)
    return df.iloc[win_has_label_idx]

windowed_df = pd.concat([window(tdcsfog_df,256),window(defog_df,200)])
tdcsfog_metadata = pd.read_csv(p + '/tdcsfog_metadata.csv')
defog_metadata = pd.read_csv(p + '/defog_metadata.csv')
metadata= pd.concat([tdcsfog_metadata,defog_metadata])
windowed_df = windowed_df.merge(metadata, on='Id', how='inner')
encoder = LabelEncoder()
windowed_df['Id']=encoder.fit_transform(windowed_df['Id'])

In [16]:
class fogDataset(Dataset):
    def __init__(self, df, ds_type=None):
        self.ds_type = ds_type
        self.df = df      
        self.features = ['AccV', 'AccML', 'AccAP','Angle']
        if ds_type == "self":
            self.labels = ['AccVAboveAverage', 'AccMLAboveAverage', 'AccAPAboveAverage']
        else:
            self.labels = ['StartHesitation', 'Turn' , 'Walking']
    
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        record = self.df.iloc[idx]
        y = torch.Tensor(record[self.labels])
        x = torch.Tensor(record[self.features])        
     
        return x,y
    
    def get_head(self):
        return self.df.head()
        

In [17]:
class dataset_build_test(Dataset):
    def __init__(self,x):
        self.id_num = x["Id"].to_numpy()
        cols = ['AccV', 'AccML', 'AccAP', 'Angle']
        self.x = torch.tensor(x[cols].to_numpy(),dtype=torch.float32)
        self.len = x.shape[0]

    def __getitem__(self,idx):
        return self.x[idx],self.id_num[idx]
  
    def __len__(self):
        return self.len

In [18]:
from sklearn.model_selection import GroupShuffleSplit
from torch.utils.data import Subset


X = windowed_df.drop(['AccV', 'AccML', 'AccAP'], axis=1)
y = windowed_df[['AccV', 'AccML', 'AccAP']]

gss = GroupShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

train_idx, val_idx = next(gss.split(X, y, windowed_df['Subject']))
train_df,val_df =windowed_df.iloc[train_idx],df.iloc[val_idx]

train_df['Angle'] = np.arctan2(train_df['AccAP'], train_df['AccV'])
val_df['Angle'] = np.arctan2(val_df['AccAP'],val_df['AccV'])

train_ds = fogDataset(train_df)
validation_ds = fogDataset(val_df)

len(train_df)/len(windowed_df),len(val_df)/len(windowed_df)

/tmp/ipykernel_34/2018393485.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['Angle'] = np.arctan2(train_df['AccAP'], train_df['AccV'])
/tmp/ipykernel_34/2018393485.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_df['Angle'] = np.arctan2(val_df['AccAP'],val_df['AccV'])


(0.5281437582443107, 0.47185624175568935)

# Naive baselines

In [ ]:
from sklearn.metrics import average_precision_score

train_class_distribution = df[['StartHesitation', 'Turn', 'Walking']].sum() / len(df)
train_naive_prediction = np.zeros_like(df[['StartHesitation', 'Turn', 'Walking']])
train_naive_prediction[:, np.argmax(train_class_distribution)] = 1
train_score = average_precision_score(df[['StartHesitation', 'Turn', 'Walking']], train_naive_prediction, average='weighted')

print("training score:", train_score)

In [19]:
from sklearn.ensemble import RandomForestClassifier, MLPClassifier
from sklearn.metrics import average_precision_score, log_loss

rfc = RandomForestClassifier(n_estimators=100, random_state=42)
rfc.fit(train_df[['AccV', 'AccML', 'AccAP', 'Angle']], train_df[['StartHesitation', 'Turn', 'Walking']])

RandomForestClassifier(random_state=42)

In [39]:
y_pred = rfc.predict(val_df[['AccV', 'AccML', 'AccAP', 'Angle']])
map_score = average_precision_score(val_df[['StartHesitation', 'Turn', 'Walking']].values, y_pred, average='weighted')
print("MAP Score:", map_score)

MAP Score: 0.27313612367581697


In [51]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_df[['AccV', 'AccML', 'AccAP', 'Angle']], train_df[['StartHesitation', 'Turn', 'Walking']])

y_pred = knn.predict(val_df[['AccV', 'AccML', 'AccAP', 'Angle']])
map_score = average_precision_score(val_df[['StartHesitation', 'Turn', 'Walking']].values, y_pred, average='weighted')
print("MAP Score:", map_score)

MAP Score: 0.27613373766795807


# Self supervised dataset

In [ ]:
def self_supervised_target(df):
    AccVthreshold = df['AccV'].mean() 
    AccMLthreshold = df['AccML'].mean() 
    AccAPthreshold = df['AccAP'].mean() 
    df['AccVAboveAverage'] = df['AccV'].apply(lambda x: 1 if x > AccVthreshold else 0)
    df['AccMLAboveAverage'] = df['AccML'].apply(lambda x: 1 if x > AccMLthreshold else 0)
    df['AccAPAboveAverage'] = df['AccAP'].apply(lambda x: 1 if x > AccAPthreshold else 0)
    return df


ss_train_df = self_supervised_target(train_df).drop(['StartHesitation', 'Turn', 'Walking', 'Id','Subject','Valid','Task'], axis=1)
ss_val_df = self_supervised_target(val_df).drop(['StartHesitation', 'Turn', 'Walking', 'Id','Subject','Valid','Task'], axis=1)

ss_train_ds = fogDataset(ss_train_df, "self")
ss_validation_ds = fogDataset(ss_val_df, "self")

# LSTM Model

In [ ]:
def train_model(model,tags,epochs,batch_size,lr):
    trainer = L.Trainer(
        callbacks=[TQDMProgressBar(refresh_rate=100)],
        accelerator='auto',
        max_epochs=epochs,
        devices=1,
    )
    run = neptune.init_run(
        project="sadotal/ML-task2",
        api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJmMjIwNjBjMS1kMDhhLTQ4MzctYmIwZS05ZDUyY2E2ZjZiM2QifQ==",)
    
    run["sys/tags"].add(tags)
    run["config/lr"]=lr
    run["config/epochs"]=trainer.max_epochs
    run["config/batch_size"]=batch_size
    model.run=run
    model.lr=lr
    trainer.fit(model,model.train_loader,model.val_loader)
    print('===vaidate===')
    trainer.validate(model,model.val_loader)
    print(trainer.callback_metrics['val_loss'])
    val_loss=trainer.callback_metrics['val_loss']

In [ ]:
class LSTM_Model(L.LightningModule):
    def __init__(self, neptun_run=None) -> None:
        super().__init__()
        self.train_loader=None
        self.val_loader=None
        self.run=neptun_run
        self.self_supervised = False
        
        self.val_accuracy = Accuracy(task="multiclass", num_classes=3)
        self.test_accuracy = Accuracy(task="multiclass", num_classes=3)
        
        self.n_features = 4
        self.hidden_size = 128
        self.seq_len = 24
        self.num_layers = 2
        self.dropout = 0.5
        self.learning_rate = 1e-4
        
#         self.reg_lambda = 0.01 # set regularization strength
        
        self.lstm = nn.LSTM(input_size=self.n_features, 
                            hidden_size=self.hidden_size,
                            num_layers=self.num_layers, 
                            dropout=self.dropout, 
                            batch_first=True)
        
        self.linear1 = nn.Linear(self.hidden_size, 64)
        self.fc = nn.Linear(64, 3)

    def forward(self, x):
        self.lstm(x) 
        lstm_out, _ = self.lstm(x)          
        lstm_out = lstm_out.reshape(-1, self.hidden_size)
        linear_out = F.relu(self.linear1(lstm_out))
#         linear_out = F.dropout(linear_out, p=self.dropout) # add dropout
        
        fc_out = self.fc(linear_out)
        output = F.log_softmax(fc_out, dim=1)

        return output
    
    
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)
     
    def training_step(self, batch, batch_idx):
        x, y = batch
        if self.self_supervised:
            y = y.squeeze()    
        y_hat = self(x)
        
        loss = F.cross_entropy(y_hat, y)
#         l2_reg = torch.tensor(0.).to(device)
#         for param in self.parameters():
#             l2_reg += torch.norm(param)
#         loss += self.reg_lambda * l2_reg
        self.run['train/loss'].append(loss.item())
        
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch       
        
        if self.self_supervised:
            y = y.squeeze()
            
        y_hat = self(x)  
        loss = F.cross_entropy(y_hat, y)
        y = y.int()
        
        accuracy=self.val_accuracy(y_hat, y).item()
        
        self.run['validation/loss'].append(loss.item())
        self.run['validation/accuracy'].append(accuracy)
        
        self.log("val_loss", loss , prog_bar=True,on_step=True)
        self.log("val_acc", accuracy , prog_bar=True,on_step=True)
        
        return loss
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        
        if self.self_supervised:
            y = y.squeeze()
            
        y_hat = self(x)
        y = y.int()
        loss = F.cross_entropy(y_hat, y)
        
        accuracy=self.test_accuracy(y_hat, y).item()
        
        self.run['test/loss'].append(loss.item())
        self.run['test/accuracy'].append(accuracy)
        
        self.log("test_loss", loss , prog_bar=True,on_step=True)
        self.log("test_acc", accuracy, prog_bar=True,on_step=True)
        
        return loss
    
    
    def change_training_type(self, train_loader, val_loader, self_supervised=False):
        self.train_loader=train_loader
        self.val_loader=val_loader
        if self_supervised:
            self.self_supervised = True
            self.fc = nn.Linear(64, 3)    
        else:
            self.fc = nn.Linear(64, 3)
            self.self_supervised = False



In [ ]:
BATCH_SIZE = 256
device='cuda'
lr=1e-4
epochs=1
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=BATCH_SIZE,shuffle=False)
val_loader = torch.utils.data.DataLoader(validation_ds, batch_size=BATCH_SIZE,shuffle=False)

# ss_train_loader = torch.utils.data.DataLoader(ss_train_ds, batch_size=BATCH_SIZE,shuffle=False)
# ss_val_loader = torch.utils.data.DataLoader(ss_validation_ds, batch_size=BATCH_SIZE,shuffle=False)

model=LSTM_Model()

# model.change_training_type(ss_train_loader,ss_val_loader, True)
# train_model(model,['Self_Supervised'],1,BATCH_SIZE,lr)

model.change_training_type(train_loader,val_loader)
train_model(model,['main_training'],epochs,BATCH_SIZE,lr)


# Tests and plots

In [ ]:
from torchmetrics import Precision

def precision(model,loader):
    precision = Precision(task="multiclass", num_classes=3).cuda()
    precision_lst=[]
    for batch_idx, (x, y) in enumerate(loader):
        y_hat = model(x)
        pred = torch.argmax(y_hat, dim=1)
        actual = torch.argmax(y, dim=1)
        precision_lst.append(precision(pred,actual))
    return sum(precision_lst)/len(precision_lst) 

In [ ]:
f'precision is {precision(model,val_loader)}'

In [ ]:
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns

def plot_count(model,loader):
    counter =  defaultdict(int)
    for batch_idx, (x, y) in enumerate(val_loader):
        y_hat = model(x)
        pred = torch.argmax(y_hat, dim=1)
        actual = torch.argmax(y, dim=1)    
        for i,c in enumerate(actual):
            if c==pred[i]:
                counter[f'{c.item()} currect']+=1
                counter[f'{c.item()} incurrect']+=0
            else:
                counter[f'{c.item()} incurrect']+=1
                counter[f'{c.item()} currect']+=0
    data = {"Category": list(counter.keys()), "Values": list(counter.values())}
    dfplt = pd.DataFrame(data)
    dfplt = dfplt.sort_values(by="Category")
    plt.figure(figsize=(12, 6))
    sns.barplot(x="Category", y="Values", data=dfplt)
    plt.show()
            


In [ ]:
plot_count(model,val_loader)